In [59]:
import json
import minsearch
import requests

from dotenv import load_dotenv
from openai import OpenAI
from qdrant_client import QdrantClient, models

In [2]:
load_dotenv()

True

In [16]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']
    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [19]:
index = minsearch.Index(
    text_fields=['question', 'text', 'section'],
    keyword_fields=['course']
)

index.fit(documents);

In [25]:
openai_client = OpenAI()

In [94]:
def search(course, query):
    boost = {
        'question': 3.0,
        'section' : 0.5
    }
    
    results = index.search(
        query=query,
        filter_dict={'course': course},
        boost_dict=boost,
        num_results=5
    )

    return results

In [95]:
def build_prompt(query, search_results):
    prompt_template = """
        You are a course teaching assistant. Answer the QUESTION based only on the CONTEXT.
        If the CONTEXT does not contain the answer output NONE.
    
        QUESTION: {query}
    
        CONTEXT: {context}
    """.strip()
    
    context = ""
    for doc in search_results:
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(query=query, context=context).strip()

    return prompt

In [96]:
def llm(prompt):
    response = openai_client.chat.completions.create(
        model = 'gpt-4.1-mini',
        messages=[{'role': 'user', 'content': prompt}]
    )
    
    return response.choices[0].message.content

In [99]:
def rag(course, query):
    search_results = search(course, query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [100]:
course = 'data-engineering-zoomcamp'
query = "how do I run kafka?"
rag(course, query)

'To run Kafka (Java producer/consumer/kstreams/etc) in the terminal, in the project directory, use the command:\n```\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nFor running Python Kafka files (like producer.py), create and activate a virtual environment first:\n```\npython -m venv env\nsource env/bin/activate      # On Windows: env\\Scripts\\activate\npip install -r ../requirements.txt\n```\nThen run your python files within that virtual environment.\n\nIf you get permission errors like "./build.sh: Permission denied", run:\n```\nchmod +x build.sh\n```\nin the appropriate directory.\n\nDocker images should be up and running before creating the virtual environment for Python Kafka files.'

## RAG with Vector Search

In [77]:
qd_client = QdrantClient("http://localhost:6333")

In [78]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [79]:
collection_name = "zoomcamp-faq"

qd_client.delete_collection(collection_name=collection_name)
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [92]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name='course',
    field_schema='keyword'
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [80]:
points = []

for i, doc in enumerate(documents):

    text = doc['question'] + ' ' + doc['text']

    point = models.PointStruct(
        id=i,
        vector=models.Document(text=text, model=model_handle),
        payload=doc
    )
    points.append(point)

In [81]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files: 100%|██████████| 5/5 [00:01<00:00,  3.74it/s]


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [89]:
course = 'data-engineering-zoomcamp'
question = "I just discovered the course. Can I still join it?"

In [105]:
def vector_search(course, question):
    print('Using vector search')
    results = []

    query_points = qd_client.query_points(
        collection_name=collection_name,
        query = models.Document(
            text=question,
            model=model_handle
        ),
        query_filter=models.Filter(
            must=[
                models.FieldCondition(
                    key='course',
                    match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=5,
        with_payload=True
    )

    for point in query_points.points:
        results.append(point.payload)

    return results

In [106]:
def v_rag(course, query):
    search_results = vector_search(course, query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [107]:
v_rag(course, question)

Using vector search


"Yes, you can still join the course even after the start date. Even if you don't register, you're still eligible to submit the homeworks. However, keep in mind there will be deadlines for turning in the final projects, so don't leave everything for the last minute."